# Business Review Sentimental Analyser

Our study cas or use is to provide a sentimental analysis that helps to review customers comments or feedback on the rating of 1 to 5.

We use an NLP bert pretrained model which can found from the link here https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment and https://metatext.io/models/nlptown-bert-base-multilingual-uncased-sentiment we scrape the comments from businesses website with beautiful soup and convert it to a dataframe which is then feed into the model to provide the ratings of scale 1 to 5 on the business.

In [1]:
#Importing all dependences
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import tensorflow as tf
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# Instansiating the NLP bert model for use into our tokenizer
model = TFAutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [7]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# Instansiating the NLP bert model for use into our tokenizer

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

c:\Users\segun\anaconda3\envs\MLDL\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\segun\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
tokens = tokenizer.encode('You built a great empire an I do hope there are visionaries to sustain it', return_tensors='pt')

In [9]:
tokens[0]

tensor([  101, 10855, 12321,   143, 11838, 13359, 10144,   151, 10154, 18763,
        10768, 10320, 17584, 30966, 10114, 10877, 41722, 10197,   102])

In [10]:
result = model(tokens)

In [11]:
result.logits

tensor([[-1.9258, -1.7160, -0.3581,  1.2074,  2.1254]],
       grad_fn=<AddmmBackward0>)

In [12]:
int(torch.argmax(result.logits))+1

5

In [13]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

### Evaluation With Business Pages

In [14]:
# Scraping the Jumia website with product about 10 reviews from the site
r = requests.get('https://www.jumia.com.ng/catalog/productratingsreviews/sku/GE779EA1BY9XTNAFAMZ/')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('-pvs')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [15]:
reviews

['This a very quick to respond wireless mouse. I love the color combination and the fact that it takes more than 72hrs to run down. Just perfect !!!',
 'Really good to use. Sleek too',
 "Long lasting battery. Durable and precise. I like it, I can't go back to using touchpad since I've got. So convenient to use.",
 'The product is excellent',
 'It works perfectly and last for many days.',
 'Works perfectly',
 "It's exactly what it is in the picture. And it doesn't make any clicking sound",
 'It good and colourful ????',
 'Long lasting battery and smooth working experience',
 'Perfect tool']

In [16]:
df1 = pd.DataFrame(np.array(reviews), columns=['review']) # Converting it to pandas dataframe

In [17]:
df1['review'].iloc[1]

'Really good to use. Sleek too'

In [18]:
df1.shape

(10, 1)

In [19]:
# Scraping a business on the yelp website of about 10 reviews
r = requests.get('https://www.yelp.com/biz/dennys-san-francisco-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('comment.')
result1 = soup.find_all('p', {'class':regex})
review1 = [result.text for result in result1]

In [20]:
sentiment_score(df1['review'].iloc[0])

5

In [21]:
df1['sentiment'] = df1['review'].apply(lambda x: sentiment_score(x[:512]))

In [22]:
df1.head()

,review,sentiment
0,This a very quick to respond wireless mouse. I...,5
1,Really good to use. Sleek too,4
2,Long lasting battery. Durable and precise. I l...,5
3,The product is excellent,5
4,It works perfectly and last for many days.,5


In [23]:
review1

["Tbh, I haven't eat at Denny's in ages. My mom and I decided to eat dinner here after our mini-shopping spree for Christmas. The ambiance has changed significantly but it brings me back to my childhood of eating Denny's at this particular location. I'm glad they're able to survive and thrive during the pandemic. It wasn't a busy and were seated immediately. We were able to choose our spot and sat closer to the kitchen and hallway to the restroom. I ordered the Plate Lickin' Fried Chicken with sides of red skin potatoes and steamed vegetables. My mom ordered the country fried steak with mashed potatoes and a side salad. Since we were in the mood for dinner but still wanted a small taste of breakfast, we ordered a side stack of pancakes. (If you don't want a full breakfast plate, just make sure to read carefully and look for the sides on the lower page of the menu). They also have Coke Zero in case you still want soda but calorie free, with refills. Props to Ryan for his above and beyon

In [24]:
df2 = pd.DataFrame(np.array(review1), columns=['review']) # Converting it to pandas dataframe

In [25]:
df2['review'].iloc[1]

"As one of the only 24 hour open places in SF, Denny's is a place that I've been to late at night (usually after clubbing) multiple times for some much needed sustenance. The inside is fairly clean, and during the times I've been we've been able to get seated pretty quickly even with a group of 5-6. The service here was alright, nothing special but also nothing bad from what I could tell. \xa0 \xa0Honestly, if you come with the expectation that it's food at a Denny's then it's not too bad. I can't speak for other reviews on this place, but during the times I've been here the food has been pretty consistent and I didn't notice anything wrong with it. I've gotten the country-fried steak and eggs, and it's a decent amount of food for the price that you pay in SF. Probably pricier compared to Denny's outside of the city, but it's nothing absurd I feel. \xa0 \xa0Overall, at least when I've been here around 2-3 am it's been a pretty decent experience and I'm fine with coming back."

In [26]:
df2.shape

(10, 1)

In [27]:
sentiment_score(df2['review'].iloc[1])

4

In [28]:
df2['sentiment'] = df2['review'].apply(lambda x: sentiment_score(x[:512]))

In [29]:
df2.head()

,review,sentiment
0,"Tbh, I haven't eat at Denny's in ages. My mom ...",4
1,"As one of the only 24 hour open places in SF, ...",4
2,Last night at 1am to pick my brother up. My wi...,5
3,Good ol' diner food. Denny's is a nostalgic pl...,4
4,Bro im not gonna tell you how bad it is if you...,1


In [30]:
import pickle

pickle.dump(model, open('BRM.pkl', 'wb'))